In [3]:
import os
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [4]:
os.chdir(r'C:\Users\Victor\Documents\Alura\ml-latest')
os.getcwd()

'C:\\Users\\Victor\\Documents\\Alura\\ml-latest'

In [82]:
#processed dataset
#pd.read_parquet("movies.parquet").to_pickle("movies.pkl").head(1000000)
#ratings.drop('timestamp',axis=1).to_parquet('ratings.parquet')

In [191]:
%%time
#movies = pd.read_parquet("movies.parquet")
#ratings = pd.read_parquet("ratings.parquet").sample(frac = 0.001)

movies = pd.read_pickle("movies.pkl")
ratings = pd.read_pickle("ratings.pkl").head(1000000)#sample(frac = 0.001)

#movies = pd.read_csv(r"C:\Users\Victor\Documents\Alura\ml-latest-small\movies.csv")
#ratings = pd.read_csv(r"C:\Users\Victor\Documents\Alura\ml-latest-small\ratings.csv")#.sample(frac = 0.001)

Wall time: 1.79 s


In [213]:
%%time
final_dataset = ratings.pivot(index='movieId',columns='userId',values='rating')
final_dataset.fillna(0,inplace=True)
no_user_voted = ratings.groupby('movieId')['rating'].agg('count')
no_movies_voted = ratings.groupby('userId')['rating'].agg('count')
final_dataset = final_dataset.loc[no_user_voted[no_user_voted > 10].index,:]
final_dataset=final_dataset.loc[:,no_movies_voted[no_movies_voted > 100].index]
final_dataset.to_pickle("FinalMoviesDataset.pkl")

Wall time: 27.2 s


In [214]:
%%time
csr_data = csr_matrix(final_dataset.values)
final_dataset.reset_index(inplace=True)
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
knn.fit(csr_data)

Wall time: 234 ms


NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [215]:
%%time
def get_movie_recommendation(movie_name):
    n_movies_to_reccomend = 10
    movie_list = movies[movies['title'].str.contains(movie_name)]  
    if len(movie_list):        
        movie_idx= movie_list.iloc[0]['movieId']
        movie_idx = final_dataset[final_dataset['movieId'] == movie_idx].index[0]
        distances , indices = knn.kneighbors(csr_data[movie_idx],n_neighbors=n_movies_to_reccomend+1)    
        rec_movie_indices = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
        recommend_frame = []
        for val in rec_movie_indices:
            movie_idx = final_dataset.iloc[val[0]]['movieId']
            idx = movies[movies['movieId'] == movie_idx].index
            recommend_frame.append({'Title':movies.iloc[idx]['title'].values[0],'Distance':val[1]})
        df = pd.DataFrame(recommend_frame,index=range(1,n_movies_to_reccomend+1))
        return df
    else:
        return "No movies found. Please check your input"
    
get_movie_recommendation('Matrix')

Wall time: 569 ms


,Title,Distance
1,Star Wars: Episode VI - Return of the Jedi (1983),0.278823
2,Pulp Fiction (1994),0.269210
3,"Lord of the Rings: The Return of the King, The...",0.258509
4,"Sixth Sense, The (1999)",0.258391
5,Gladiator (2000),0.251706
6,Star Wars: Episode IV - A New Hope (1977),0.249922
7,Star Wars: Episode V - The Empire Strikes Back...,0.249899
8,"Lord of the Rings: The Two Towers, The (2002)",0.238238
9,Fight Club (1999),0.215279
10,"Lord of the Rings: The Fellowship of the Ring,...",0.213009


In [216]:
import pickle
pickle.dump(knn, open('knn_model.sav','wb'))

In [219]:
modelo = pickle.load(open('knn_model.sav','rb'))
MovieDataset = pd.read_pickle("FinalMoviesDataset.pkl")
csr_data = csr_matrix(MovieDataset.values)
MovieDataset.reset_index(inplace=True)

In [220]:

def get_movie_recommendation(movie_name):
    n_movies_to_reccomend = 10
    movie_list = movies[movies['title'].str.contains(movie_name)]
    if len(movie_list):        
        movie_idx= movie_list.iloc[0]['movieId']
        movie_idx = MovieDataset[MovieDataset['movieId'] == movie_idx].index[0]
        distances , indices = modelo.kneighbors(csr_data[movie_idx],n_neighbors=n_movies_to_reccomend+1)
        rec_movie_indices = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
        recommend_frame = []
        for val in rec_movie_indices:
            movie_idx = MovieDataset.iloc[val[0]]['movieId']
            idx = movies[movies['movieId'] == movie_idx].index
            recommend_frame.append({'Title':movies.iloc[idx]['title'].values[0],'Distance':val[1]})
        df = pd.DataFrame(recommend_frame,index=range(1,n_movies_to_reccomend+1))
        return df
    else:
        return "No movies found. Please check your input"

get_movie_recommendation('Matrix')

,Title,Distance
1,Star Wars: Episode VI - Return of the Jedi (1983),0.278823
2,Pulp Fiction (1994),0.269210
3,"Lord of the Rings: The Return of the King, The...",0.258509
4,"Sixth Sense, The (1999)",0.258391
5,Gladiator (2000),0.251706
6,Star Wars: Episode IV - A New Hope (1977),0.249922
7,Star Wars: Episode V - The Empire Strikes Back...,0.249899
8,"Lord of the Rings: The Two Towers, The (2002)",0.238238
9,Fight Club (1999),0.215279
10,"Lord of the Rings: The Fellowship of the Ring,...",0.213009


In [ ]:
#usar apenas o CSR matrix e nao o moviedataset

In [147]:
#https://grouplens.org/datasets/movielens/latest/


In [179]:
final_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7202 entries, 0 to 7201
Columns: 2423 entries, movieId to 10059
dtypes: float64(2422), int64(1)
memory usage: 133.1 MB


In [208]:
final_dataset.reset_index(inplace=False).shape

(7202, 2423)

In [206]:
final_dataset.values.shape

(7202, 2422)

In [203]:
csr_matrix(final_dataset.values).shape

(7202, 2422)

In [212]:
MovieDataset.reset_index(inplace=False).shape

(7202, 2424)